In [2]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KernelDensity
import matplotlib.pylab as plt
from scipy.optimize import minimize
from scipy.linalg import block_diag
from sklearn.covariance import LedoitWolf

#snippet 2.1
#Marcenko-Pastur pdf
#q=T/N 
def mpPDF(var, q, pts):
    eMin, eMax = var*(1-(1./q)**.5)**2, var*(1+(1./q)**.5)**2 # calc lambda_minus, lambda_plus
    eVal = np.linspace(eMin, eMax, pts) #Return evenly spaced numbers over a specified interval. eVal='lambda'
    #Note: 1.0/2*2 = 1.0 not 0.25=1.0/(2*2)
    pdf = q/(2*np.pi*var*eVal)*((eMax-eVal)*(eVal-eMin))**.5 #np.allclose(np.flip((eMax-eVal)), (eVal-eMin))==True
    pdf = pd.Series(pdf, index=eVal)
    return pdf

#snippet 2.2
#Test Marcenko-Pastur Thm
def getPCA(matrix):
    # Get eVal, eVec from a Hermitian matrix
    eVal, eVec = np.linalg.eig(matrix) #complex Hermitian (conjugate symmetric) or a real symmetric matrix.
    indices = eVal.argsort()[::-1] #arguments for sorting eval desc
    eVal,eVec = eVal[indices],eVec[:,indices]
    eVal = np.diagflat(eVal) # identity matrix with eigenvalues as diagonal
    return eVal,eVec
    
def fitKDE(obs, bWidth=.15, kernel='gaussian', x=None):
    #Fit kernel to a series of obs, and derive the prob of obs
    # x is the array of values on which the fit KDE will be evaluated
    #print(len(obs.shape) == 1)
    if len(obs.shape) == 1: obs = obs.reshape(-1,1)
    kde = KernelDensity(kernel = kernel, bandwidth = bWidth).fit(obs)
    #print(x is None)
    if x is None: x = np.unique(obs).reshape(-1,1)
    #print(len(x.shape))
    if len(x.shape) == 1: x = x.reshape(-1,1)
    logProb = kde.score_samples(x) # log(density)
    pdf = pd.Series(np.exp(logProb), index=x.flatten())
    return pdf

#snippet 2.3
def getRndCov(nCols, nFacts): #nFacts - contains signal out of nCols
    w = np.random.normal(size=(nCols, nFacts))
    cov = np.dot(w, w.T) #random cov matrix, however not full rank
    cov += np.diag(np.random.uniform(size=nCols)) #full rank cov
    return cov

def cov2corr(cov):
    # Derive the correlation matrix from a covariance matrix
    std = np.sqrt(np.diag(cov))
    corr = cov/np.outer(std,std)
    corr[corr<-1], corr[corr>1] = -1,1 #for numerical errors
    return corr
    
def corr2cov(corr, std):
    cov = corr * np.outer(std, std)
    return cov     
    
#snippet 2.4 - fitting the marcenko-pastur pdf - find variance
#Fit error
def errPDFs(var, eVal, q, bWidth, pts=1000):
    var = var[0]
    pdf0 = mpPDF(var, q, pts) #theoretical pdf
    pdf1 = fitKDE(eVal, bWidth, x=pdf0.index.values) #empirical pdf
    sse = np.sum((pdf1-pdf0)**2)
    print("sse:"+str(sse))
    return sse 
    
# find max random eVal by fitting Marcenko's dist
# and return variance
def findMaxEval(eVal, q, bWidth):
    out = minimize(lambda *x: errPDFs(*x), x0=np.array(0.5), args=(eVal, q, bWidth), bounds=((1E-5, 1-1E-5),))
    print("found errPDFs"+str(out['x'][0]))
    if out['success']: var = out['x'][0]
    else: var=1
    eMax = var*(1+(1./q)**.5)**2
    return eMax, var
    
# code snippet 2.5 - denoising by constant residual eigenvalue
# Remove noise from corr by fixing random eigenvalue
# Operation invariante to trace(Correlation)
# The Trace of a square matrix is the _Sum_ of its eigenvalues
# The Determinate of thematrix is the _Product_ of its eigenvalues
def denoisedCorr(eVal, eVec, nFacts):
    eVal_ = np.diag(eVal).copy()
    eVal_[nFacts:] = eVal_[nFacts:].sum()/float(eVal_.shape[0] - nFacts) #all but 0..i values equals (1/N-i)sum(eVal_[i..N]))
    eVal_ = np.diag(eVal_) #square matrix with eigenvalues as diagonal: eVal_.I
    corr1 = np.dot(eVec, eVal_).dot(eVec.T) #Eigendecomposition of a symmetric matrix: S = QΛQT
    corr1 = cov2corr(corr1) # Rescaling the correlation matrix to have 1s on the main diagonal
    return corr1
    
# code snippet 2.6 - detoning
# ref: mlfinlab/portfolio_optimization/risk_estimators.py
# This method assumes a sorted set of eigenvalues and eigenvectors.
# The market component is the first eigenvector with highest eigenvalue.
# it returns singular correlation matrix: 
# "the detoned correlation matrix is singualar, as a result of eliminating (at least) one eigenvector."
# Page 32
def detoned_corr(corr, eigenvalues, eigenvectors, market_component=1):
    """
    De-tones the de-noised correlation matrix by removing the market component.
    The input is the eigenvalues and the eigenvectors of the correlation matrix and the number
    of the first eigenvalue that is above the maximum theoretical eigenvalue and the number of
    eigenvectors related to a market component.
    :param corr: (np.array) Correlation matrix to detone.
    :param eigenvalues: (np.array) Matrix with eigenvalues on the main diagonal.
    :param eigenvectors: (float) Eigenvectors array.
    :param market_component: (int) Number of fist eigevectors related to a market component. (1 by default)
    :return: (np.array) De-toned correlation matrix.
    """
    
    # Getting the eigenvalues and eigenvectors related to market component
    eigenvalues_mark = eigenvalues[:market_component, :market_component]
    eigenvectors_mark = eigenvectors[:, :market_component]
    
    # Calculating the market component correlation
    corr_mark = np.dot(eigenvectors_mark, eigenvalues_mark).dot(eigenvectors_mark.T)
    
    # Removing the market component from the de-noised correlation matrix
    corr = corr - corr_mark
    
    # Rescaling the correlation matrix to have 1s on the main diagonal
    corr = cov2corr(corr)
    
    return corr
            

In [4]:
x=np.random.normal(size=(10000,1000))
eVal0,eVec0=getPCA(np.corrcoef(x,rowvar=0))

In [6]:
alpha,nCols,nFact,q=.995,1000,100,10
cov=np.cov(np.random.normal(size=(nCols*q,nCols)),rowvar=0)
cov=alpha*cov+(1-alpha)*getRndCov(nCols,nFact)
corr0=cov2corr(cov)
eVal0,eVec0=getPCA(corr0)

In [7]:
eMax0,var0=findMaxEval(np.diag(eVal0),q,bWidth=.01)
nFacts0=eVal0.shape[0]-np.diag(eVal0)[::-1].searchsorted(eMax0)

sse:752.5686337686157
sse:752.5685383945494
sse:285.1467340303001
sse:285.14673035987647
sse:277.7181116542274
sse:277.71811611273864
sse:232.60059342987353
sse:232.6006006348726
sse:119.55595426384266
sse:119.55592535473374
sse:182.2245078176563
sse:182.2245172623255
sse:19.104584621773945
sse:19.104587893842822
sse:7036642712506.261
sse:7022590508897.954
sse:1223.8926644103444
sse:1223.8925272992533
sse:17.543335395760295
sse:17.5433352365119
sse:285.1467340303001
sse:285.14673035987647
sse:17.55079656472821
sse:17.55079684856071
sse:17.539965550600257
sse:17.539965550604325
sse:17.539965550600005
sse:17.539965550600037
found errPDFs0.6792503801855408


In [57]:
eMax0

1.176771078788418

In [8]:
eVal,eVec,nFacts=eVal0,eVec0,nFacts0

In [46]:
eVal_ = np.diag(eVal).copy()
eVal_[nFacts:] = eVal_[nFacts:].sum()/float(eVal_.shape[0] - nFacts) #all but 0..i values equals (1/N-i)sum(eVal_[i..N]))
eVal_ = np.diag(eVal_) #square matrix with eigenvalues as diagonal: eVal_.I
corr1 = np.dot(eVec, eVal_).dot(eVec.T) #Eigendecomposition of a symmetric matrix: S = QΛQT
corr1 = cov2corr(corr1) # Rescaling the correlation matrix to have 1s on the main diagonal

In [67]:
np.diag(eVal_)[95:140]

array([2.46388994, 2.42963544, 2.40618884, 2.33549868, 2.32081352,
       0.66805344, 0.66805344, 0.66805344, 0.66805344, 0.66805344,
       0.66805344, 0.66805344, 0.66805344, 0.66805344, 0.66805344,
       0.66805344, 0.66805344, 0.66805344, 0.66805344, 0.66805344,
       0.66805344, 0.66805344, 0.66805344, 0.66805344, 0.66805344,
       0.66805344, 0.66805344, 0.66805344, 0.66805344, 0.66805344,
       0.66805344, 0.66805344, 0.66805344, 0.66805344, 0.66805344,
       0.66805344, 0.66805344, 0.66805344, 0.66805344, 0.66805344,
       0.66805344, 0.66805344, 0.66805344, 0.66805344, 0.66805344])

In [ ]:
np.diag(eVal0)[95:140]

array([2.46388994, 2.42963544, 2.40618884, 2.33549868, 2.32081352,
       1.14129191, 1.12835373, 1.12103697, 1.11543288, 1.11281898,
       1.10528223, 1.10466904, 1.10021575, 1.09867869, 1.09735789,
       1.09414669, 1.09162142, 1.0876504 , 1.08521577, 1.08244341,
       1.08089042, 1.07839975, 1.07701641, 1.07370922, 1.0732285 ,
       1.06900785, 1.06617465, 1.0656383 , 1.06370756, 1.06259816,
       1.05955891, 1.05735919, 1.05539981, 1.05344286, 1.05282789,
       1.05014248, 1.04899392, 1.04811084, 1.04385529, 1.04284107,
       1.04225321, 1.03944917, 1.0382955 , 1.03694818, 1.03571676])

In [72]:
np.diag(eVal0)[-50:]

array([0.37966198, 0.37923144, 0.37828769, 0.37696857, 0.37552142,
       0.37487405, 0.37460198, 0.37376679, 0.37316377, 0.37201607,
       0.37062554, 0.3700061 , 0.36990281, 0.36848378, 0.3676012 ,
       0.36701735, 0.36633318, 0.36513747, 0.36458364, 0.36446073,
       0.36376986, 0.36275384, 0.36220663, 0.36110158, 0.36010413,
       0.35918187, 0.35830395, 0.35708041, 0.35545435, 0.35468765,
       0.35426249, 0.35310368, 0.35219944, 0.35108808, 0.35058677,
       0.34876921, 0.34765135, 0.34728467, 0.34533212, 0.34495195,
       0.34387426, 0.34358347, 0.34097509, 0.34038054, 0.33864305,
       0.33781737, 0.337065  , 0.33649863, 0.33047496, 0.32727138])

In [73]:
np.diag(eVal_)[-50:]

array([0.66805344, 0.66805344, 0.66805344, 0.66805344, 0.66805344,
       0.66805344, 0.66805344, 0.66805344, 0.66805344, 0.66805344,
       0.66805344, 0.66805344, 0.66805344, 0.66805344, 0.66805344,
       0.66805344, 0.66805344, 0.66805344, 0.66805344, 0.66805344,
       0.66805344, 0.66805344, 0.66805344, 0.66805344, 0.66805344,
       0.66805344, 0.66805344, 0.66805344, 0.66805344, 0.66805344,
       0.66805344, 0.66805344, 0.66805344, 0.66805344, 0.66805344,
       0.66805344, 0.66805344, 0.66805344, 0.66805344, 0.66805344,
       0.66805344, 0.66805344, 0.66805344, 0.66805344, 0.66805344,
       0.66805344, 0.66805344, 0.66805344, 0.66805344, 0.66805344])

In [74]:
# bu yukardaki 4 outputta eigen valulara bakıyoruz correction işlemi ne gibi bir değişiklik yapıyor iyice anlayabilmek için
# emaxtan kucuk olanların hepsini alıp topluyor sonra kaç tanesi öyleyse onların sayısına bölüyor
# cıkan değeri hepsine o değeri atıyor. adı buradan gelıyor constant residual 

In [ ]:
np.diag(eVal0)[95:140]

In [ ]:
def denoisedCorr(eVal, eVec, nFacts):
    eVal_ = np.diag(eVal).copy()
    eVal_[nFacts:] = eVal_[nFacts:].sum()/float(eVal_.shape[0] - nFacts) #all but 0..i values equals (1/N-i)sum(eVal_[i..N]))
    eVal_ = np.diag(eVal_) #square matrix with eigenvalues as diagonal: eVal_.I
    corr1 = np.dot(eVec, eVal_).dot(eVec.T) #Eigendecomposition of a symmetric matrix: S = QΛQT
    corr1 = cov2corr(corr1) # Rescaling the correlation matrix to have 1s on the main diagonal
    return corr1